In [1]:
# System
import os
from dotenv import load_dotenv
import tempfile
from titlecase import titlecase

# LLM Models
from langchain_openai import ChatOpenAI
from openai import OpenAI

# Template
from langchain_core.prompts import ChatPromptTemplate

# OutputParsers
from langchain.schema.output_parser import StrOutputParser

# Gradio frontend
import gradio as gr

In [2]:
load_dotenv()

True

In [3]:
client = OpenAI()

In [4]:
chat_model = ChatOpenAI(model="gpt-4o-mini-2024-07-18",
                        max_completion_tokens=2048,
                        api_key=os.getenv("OPENAI_API_KEY"),
                        temperature=0.4)

In [5]:
books = {"A Christmas Carol": '.\\data\\A_Christmas_Carol.txt',
         "A Farewell to Arms": '.\\data\\A_Farewell_to_Arms.txt',
         "A Tale of Two Cities": '.\\data\\A_Tale_of_Two_Cities.txt',
         "Adventures of Huckleberry Finn": '.\\data\\Adventures_of_Huckleberry_Finn.txt',
         "Alice's Adventures in Wonderland": '.\\data\\Alice_Adventures_in_Wonderland.txt',
         "All Quiet on the Western Front": '.\\data\\All_Quiet_on_the_Western_Front.txt',
         "Anna Karenina": '.\\data\\Anna_Karenina.txt',
         "Crime and Punishment": '.\\data\\Crime_and_Punishment.txt',
         "David Copperfield": '.\\data\\David_Copperfield.txt',
         "Emma": '.\\data\\Emma.txt',
         "Great Expectations": '.\\data\\Great_Expectations.txt',
         "Heart of Darkness": '.\\data\\Heart_of_Darkness.txt',
         "Jane Eyre": '.\\data\\Jane_Eyre.txt',
         "Macbeth": '.\\data\\Macbeth.txt',
         "Moby-Dick": '.\\data\\Moby_Dick.txt',
         "Oliver Twist": '.\\data\\Oliver_Twist.txt',
         "Personal Memoirs of U. S. Grant": '.\\data\\Personal_Memoirs.txt',
         "Pride and Prejudice": '.\\data\\Pride_and_Prejudice.txt',
         "Sense and Sensibility": '.\\data\\Sense_and_Sensibility.txt',
         "Silas Marner": '.\\data\\Silas_Marner.txt',
         "The Great Gatsby": '.\\data\\The_Great_Gatsby.txt',
         "The Hound of the Baskervilles": '.\\data\\The_Hound_of_the_Baskervilles.txt',
         "The Jungle": '.\\data\\The_Jungle.txt',
         "The Red Badge of Courage": '.\\data\\The_Red_Badge_of_Courage.txt',
         "The Sun Also Rises": '.\\data\\The_Sun_Also_Rises.txt',
         "Treasure Island": '.\\data\\Treasure_Island.txt',
         "Wuthering Heights": '.\\data\\Wut\\hering_Heights.txt'
         }

In [6]:
def book_headline(book_title="Moby-Dick"):
    filepath = books[book_title]

    with open(file=filepath, mode='r') as file:
        f = file.read()

    f_lines = f.split('\n')
    title = titlecase(f_lines[0])
    author = f_lines[2].title()
    chapters = f.split('[divider]')[1:]
    num_chapters = len(chapters)
    headline = f'{title} {author} \nNumber of Chapters: {num_chapters}'.replace(' By ', ' by ')

    return headline, None, None

In [7]:
def english_language_translation(text_input: str = 'Hi, how are you?', language: str = 'Chinese'):

    response = chat_model.invoke(f"Translate the following words in English to traditional {language}. "
                                 f"Give only the {language} translation: {text_input}")

    return response.content

In [8]:
def process_audio(audio):
    # Load the audio file
    input_file_path = audio  # This is the path to the uploaded file
    output_file_path = "custom_output_filename.wav"  # Desired output filename

    # Simulate processing (e.g., copying the file)
    os.rename(input_file_path, output_file_path)

    # Return the new file path
    return output_file_path

In [9]:
def book_chapter_summary(book_title="Moby Dick", chapter_num=1, language='English', voice_gender='Male', speed=1.0):
    filepath = books[book_title]

    with open(file=filepath, mode='r') as file:
        f = file.read()

    f_lines = f.split('\n')
    title = titlecase(f_lines[0])
    author = f_lines[2].title()
    chapters = f.split('[divider]')[1:]

    num_chapters = len(chapters)

    if chapter_num > num_chapters:
        chapter_selected = num_chapters
    else:
        chapter_selected = chapter_num

    chapter = chapters[chapter_selected - 1]

    prompt = ChatPromptTemplate.from_template(
        """
        Provide a summary of the following text, at most 200 words. Answer must be based only on the text given. 
        Context: {chapter}
        """
    )

    chat_chain = prompt | chat_model | StrOutputParser()

    response = chat_chain.invoke({"chapter": chapters[chapter_num - 1]})

    if language == 'English':
        response1 = response
    else:
        response1 = english_language_translation(text_input=response, language=language)

    if voice_gender == 'Male':
        voice_model = 'onyx'
    else:
        voice_model = 'coral'

    audio = client.audio.speech.create(input=response1,
                                       model="gpt-4o-mini-tts",
                                       voice=voice_model,
                                       instructions="Speak in a story telling tone.",
                                       response_format="mp3",
                                       speed=speed)

    prefix = f"{book_title.replace(' ', '_')}_Chapter_{chapter_selected}_{language}_"

    with tempfile.NamedTemporaryFile(delete=False, suffix=".mp3", prefix=prefix) as temp_audio_file:
        temp_audio_file.write(audio.content)
        temp_file_path = temp_audio_file.name

    headline = f'{title} {author} \nChapter {chapter_selected} of {num_chapters}'.replace(' By ', ' by ')

    return headline, response1, temp_file_path

In [10]:
css_template = """                   
               h1 {
                   font-size: 36px;
                   font-family: Verdana, Arial, Sans-Serif;
                   font-style: normal;
                   font-weight: bold;
                   # color: black; # rgb(238, 130, 238)
                   # background-color: yellow;
                   text-align: center;
                   text-transform: normal;
                   letter-spacing: 2px;
                   # border: 1px solid lightblue;
                   # border-radius: 0px
                   }

               p {
                   font-size: 14px;
                   font-family: Times New Roman, Times;
                   font-style: normal;
                   font-weight: normal;
                   # color: black; # rgb(238, 130, 238)
                   # background-color: yellow;
                   text-align: left;
                   text-transform: normal;
                   letter-spacing: 0px;
                   # border: 1px solid lightblue;
                   # border-radius: 0px
                   }
               """

In [11]:
with gr.Blocks(theme=gr.themes.Base(),
               title='Book Summary by Chapter',
               css=css_template) as interface:
    gr.Markdown("# Book Summary by Chapter", elem_id='title')
    gr.Markdown("This app provides concise summaries of chapters from classic novels and plays. "
                "Simply select the book and chapter, choose your preferred output language, "
                "and click 'Submit' to generate the summary. You can then click 'Play' to listen to the audio version, "
                "and there's also an option to download the summary as an MP3 file. "
                "Enjoy exploring literature in a new way!")
    with gr.Row():  # Create a row to hold two columns
        with gr.Column():  # First column
            dropdown2 = gr.Dropdown(choices=list(books.keys()),
                                    value='A Christmas Carol',
                                    multiselect=False,
                                    label="Title of the Book")
            number1 = gr.Number(label="Chapter Number",
                                value=1,
                                minimum=1)
            dropdown3 = gr.Dropdown(choices=['English', 'Chinese', 'Spanish', 'French'],
                                    value='English',
                                    multiselect=False,
                                    label="Language Output")
            dropdown4 = gr.Dropdown(choices=['Male', 'Female'],
                                    value='Male',
                                    multiselect=False,
                                    label="Speaker Gender")
            slider1 = gr.Slider(minimum=0.80,
                                maximum=1.20,
                                value=1.0,
                                step=0.01,
                                label="Speaker Speed")
            button1 = gr.Button(value="Submit")
        with gr.Column():  # Second column
            output1 = gr.Text(label="Book Headline")
            output2 = gr.Text(label="Chapter Summary")
            output3 = gr.Audio(label="Generated Audio",
                               type="filepath",
                               autoplay=False,
                               show_share_button=False)
    button1.click(fn=book_chapter_summary,
                  inputs=[dropdown2, number1, dropdown3, dropdown4, slider1],
                  outputs=[output1, output2, output3])
    dropdown2.select(fn=book_headline,
                     inputs=[dropdown2],
                     outputs=[output1, output2, output3])

In [12]:
if __name__ == '__main__':
    interface.launch(share=False)

* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.


In [13]:
interface.close()

Closing server running on port: 7860
